![Imgur](https://i.imgur.com/5pXzCIu.png)

# Data Science va Sun'iy Intellekt Praktikum

## 5-MODUL. Machine Learning

### Portfolio uchun vazifa: Toshkent shahrida uylarning narxini aniqlash.

Ushbu amaliyotda sizning vazifangiz berilgan ma`lumotlar asosida Toshkent shahridagi uylarning narxini aniqlash.

In [36]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/anvarnarz/praktikum_datasets/main/housing_data_08-02-2021.csv')
df.head()

,location,district,rooms,size,level,max_levels,price
0,"город Ташкент, Юнусабадский район, Юнусабад 8-...",Юнусабадский,3,57,4,4,52000
1,"город Ташкент, Яккасарайский район, 1-й тупик ...",Яккасарайский,2,52,4,5,56000
2,"город Ташкент, Чиланзарский район, Чиланзар 2-...",Чиланзарский,2,42,4,4,37000
3,"город Ташкент, Чиланзарский район, Чиланзар 9-...",Чиланзарский,3,65,1,4,49500
4,"город Ташкент, Чиланзарский район, площадь Актепа",Чиланзарский,3,70,3,5,55000


# Ustunlar ta'rifi
- `location` - sotilayotgan uy manzili
- `district` - uy joylashgan tuman
- `rooms` - xonalar soni
- `size` - uy maydoni (kv.m)
- `level` - uy joylashgan qavat
- `max_levels` - ja'mi qavatlar soni
- `price` - uy narxi

## Vazifani CRSIP-DM Metolodgiyasi yordamida bajaring.
<img src="https://i.imgur.com/dzZnnYi.png" alt="CRISP-DM" width="800"/>

In [37]:
import pandas as pd
import numpy as np

df = pd.read_csv('/content/drive/MyDrive/Projects/ToshkentUyNarxlari/full_dataset.csv', index_col=0)
df

,district,rooms,size,level,max_levels,price
0,Юнусабадский,3,57.0,4,4,52000.0
1,Яккасарайский,2,52.0,4,5,56000.0
2,Чиланзарский,2,42.0,4,4,37000.0
3,Чиланзарский,3,65.0,1,4,49500.0
4,Чиланзарский,3,70.0,3,5,55000.0
...,...,...,...,...,...,...
7560,Яшнободский,1,38.0,5,5,24500.0
7561,Яшнободский,2,49.0,1,4,32000.0
7562,Шайхантахурский,2,64.0,3,9,40000.0
7563,Мирзо-Улугбекский,1,18.0,1,4,11000.0


In [38]:
x_train = df.drop('price', axis=1)
x_num_train = x_train.drop('district', axis=1)
y = df[['price']]

In [39]:
from sklearn.base import BaseEstimator, TransformerMixin
# bizga kerak ustunlar indekslari
rooms_ix, size_ix = 1, 2

class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self, add_size_per_room = True):
        self.add_size_per_room = add_size_per_room
    def fit(self, X, y=None):
        return self # bizni funksiyamiz faqat transformer. estimator emas
    def transform(self, X):
        size_per_rooms = X[:, size_ix] / X[:, rooms_ix]
        return np.c_[X, size_per_rooms]


In [40]:
from sklearn.impute import SimpleImputer
simple_imputer = SimpleImputer(strategy='median')
y = simple_imputer.fit_transform(y)

In [41]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('add_atributes', CombinedAttributesAdder()),
    ('std_scaler', StandardScaler())
])

In [42]:
num_pipeline.fit_transform(x_num_train)

array([[ 0.34498717, -0.03773199,  0.13442554, -0.77999447,  0.14029711],
       [-0.57656225, -0.04108519,  0.13442554, -0.397307  ,  0.22245989],
       [-0.57656225, -0.04779158,  0.13442554, -0.77999447,  0.44547313],
       ...,
       [-0.57656225, -0.03303752, -0.3125763 ,  1.13344291, -0.14342122],
       [-1.49811166, -0.06388692, -1.20657998, -0.77999447, -0.03772223],
       [-1.49811166, -0.05583925, -0.75957814, -0.77999447,  0.09757247]])

In [43]:
from sklearn.compose import ColumnTransformer

num_attribs = list(x_num_train)
cat_attribs = ['district']

full_pipeline = ColumnTransformer([
    ('num', num_pipeline, num_attribs),
    ('cat', OneHotEncoder(), cat_attribs)
])

In [44]:
x_prepared = full_pipeline.fit_transform(x_train)
x_prepared[:5]

array([[ 0.34498717, -0.03773199,  0.13442554, -0.77999447,  0.14029711,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  1.        ,  0.        ,
         0.        ,  0.        ],
       [-0.57656225, -0.04108519,  0.13442554, -0.397307  ,  0.22245989,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  1.        ,
         0.        ,  0.        ],
       [-0.57656225, -0.04779158,  0.13442554, -0.77999447,  0.44547313,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  1.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ],
       [ 0.34498717, -0.03236688, -1.20657998, -0.77999447, -0.52686462,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  1.        ,  0.        ,  0.        ,  0.        ,
         0.        

# ML model yaratish


In [45]:
from sklearn.linear_model import LinearRegression

LR_model = LinearRegression()
LR_model.fit(x_prepared, y)

LinearRegression()

In [46]:
from sklearn.model_selection import cross_val_score

In [47]:
def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Std.dev:", scores.std())

In [48]:
scores = cross_val_score(LR_model, x_prepared, y, scoring="neg_mean_squared_error", cv=10)
LR_rmse_scores = np.sqrt(-scores)

In [49]:
display_scores(LR_rmse_scores)

Scores: [  72013.70803641   46185.47995094  144506.62158078   42221.92448309
   49148.44071801   43000.57578624   51568.43386282 1888810.74194482
  165655.47099545  645841.16897623]
Mean: 314895.2566334793
Std.dev: 552944.4992405677


In [69]:
from sklearn.ensemble import RandomForestRegressor
RF_model = RandomForestRegressor()
RF_model.fit(x_prepared, y)

<ipython-input-69-677fef979c9d>:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  RF_model.fit(x_prepared, y)


RandomForestRegressor()

In [70]:
scores = cross_val_score(RF_model, x_prepared, y, scoring="neg_mean_squared_error", cv=10)
RF_rmse_scores = np.sqrt(-scores)

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y was passed whe

In [72]:
display_scores(RF_rmse_scores)

Scores: [  86026.53615812   84297.94397846  262694.97413184   43115.43382719
  403304.30014205  569295.27122489  185228.31720167 1889625.63068206
  164692.51298461  651976.86848849]
Mean: 434025.7788819381
Std.dev: 523978.4356119191


In [80]:
df['district'].unique()

array(['Юнусабадский', 'Яккасарайский', 'Чиланзарский', 'Учтепинский',
       'Мирзо-Улугбекский', 'Яшнободский', 'Мирабадский',
       'Шайхантахурский', 'Олмазорский', 'Сергелийский', 'Янгихаётский',
       'Бектемирский'], dtype=object)

In [81]:
indexes = []
for i in df['district'].unique():
    df2 = df[df['district']==i]
    indexes.append(df2.index[0])
indexes

[0, 1, 2, 8, 18, 23, 25, 61, 69, 147, 227, 1559]

In [83]:
test = x_train.loc[indexes]
test

,district,rooms,size,level,max_levels
0,Юнусабадский,3,57.0,4,4
1,Яккасарайский,2,52.0,4,5
2,Чиланзарский,2,42.0,4,4
8,Учтепинский,2,51.0,3,4
18,Мирзо-Улугбекский,3,65.0,1,4
23,Яшнободский,2,70.0,5,5
25,Мирабадский,3,80.0,2,4
61,Шайхантахурский,4,93.0,4,4
69,Олмазорский,2,57.0,3,4
147,Сергелийский,3,69.0,2,9


In [84]:
test_p = full_pipeline.fit_transform(test)
test_p

array([[ 0.65094455, -0.54374884,  0.24426598, -0.66617339,  0.46491401,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  1.        ,  0.        ,
         0.        ,  0.        ],
       [-0.91132238, -0.91448668,  0.24426598, -0.13323468,  0.71814685,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  1.        ,
         0.        ,  0.        ],
       [-0.91132238, -1.65596237,  0.24426598, -0.66617339,  1.40549311,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  1.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ],
       [-0.91132238, -0.98863425, -0.34197238, -0.66617339,  0.03888701,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         1.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        

In [92]:
RF_model.predict(test_p)

array([458399.65666667,  36569.07333333,  35213.98      ,  34457.99      ,
       145941.        , 175438.20666667,  82593.        , 127748.5       ,
        34845.99      , 137032.75      ,  34576.98      , 187407.83      ])

In [91]:
df.loc[indexes]['price'].copy()

0       52000.0
1       56000.0
2       37000.0
8       26200.0
18      32500.0
23      47000.0
25      76000.0
61      63500.0
69      42000.0
147     40000.0
227     36000.0
1559    33000.0
Name: price, dtype: float64